In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [3]:
df = pd.read_csv(
    "/content/pltr_ts_processed.csv",
    index_col=0,
    parse_dates=True
)

ret = df["log_return"].dropna().values

In [4]:
def create_sequences(series, window):
    X, y = [], []
    for i in range(window, len(series)):
        X.append(series[i-window:i])
        y.append(series[i])
    return np.array(X), np.array(y)

WINDOW = 20

X_seq, y_seq = create_sequences(ret, WINDOW)

X_seq = X_seq.reshape(X_seq.shape[0], X_seq.shape[1], 1)


In [5]:
train_size = int(len(X_seq) * 0.7)

X_train = X_seq[:train_size]
y_train = y_seq[:train_size]

X_test  = X_seq[train_size:]
y_test  = y_seq[train_size:]


In [6]:
model = Sequential([
    LSTM(32, input_shape=(WINDOW, 1)),
    Dense(1)
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="mse"
)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)


Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 0.0023 - val_loss: 0.0016
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0025 - val_loss: 0.0016
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0020 - val_loss: 0.0016
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0021 - val_loss: 0.0016
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0021 - val_loss: 0.0016
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0020 - val_loss: 0.0016
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0020 - val_loss: 0.0016
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0020 - val_loss: 0.0016
Epoch 9/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0021 - val_loss: 0.0017
Epoch 10/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0023 - val_loss: 0.0017
Epoch 11/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0022 - val_loss: 0.0017
Epoch 12/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0

In [8]:
dl_preds = model.predict(X_test).ravel()

dl_rmse = np.sqrt(mean_squared_error(y_test, dl_preds))
dl_rmse

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step


np.float64(0.041374305552240545)

In [10]:
print("LSTM RMSE:", dl_rmse)

LSTM RMSE: 0.041374305552240545


In [11]:
pd.DataFrame({
    "actual": y_test,
    "dl_pred": dl_preds
}).to_csv("/content/dl_wfo_predictions.csv", index=False)